# Treinar nosso próprio modelo de classificação de Redações

# Dataset

In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset

In [7]:
dados_redacoes = load_dataset('csv', data_files='/content/redacoes.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dados_redacoes

DatasetDict({
    train: Dataset({
        features: ['essay', 'score'],
        num_rows: 4570
    })
})

In [9]:
dados_redacoes['train'].features

{'essay': Value(dtype='string', id=None),
 'score': Value(dtype='int64', id=None)}

In [10]:
dados_redacoes['train']['essay'][0]

'A convivência com o ensino em uma escola pública , nos mostra \\xa0como a reforma do ensino médio é necessária, com o ensino precário, escolas quase desabando, falta de educadores,a formação de estudantes fica a \\xa0cada dia mais difícil., Atualmente o índice de jovens que procuram uma educação de qualidade é maior do que em todos os séculos , temos em vista que todos procuram a melhoria, tanto na estrutura da instituição como no ensino, muitos sonham em ser: médicos, professores,advogados, contadores, e assim sucessivamente., A solução seria a constatação de uma lei em prol dos estudantes, precisamos ser ouvidos, a voz dos jovens é a voz do futuro, queremos o ensino superior, queremos oportunidades em escolas públicas, escolas bem estruturada e profissionais competentes., Creio \\xa0que a reforma deveria ocorrer, pois o intuito é a melhoria (Que tipo de melhoria?,\\xa0assim todos os jovens teriam a oportunidade de serem profissionais de sucesso. O incentivo ( aos jovens é o que torn

In [11]:
dados_redacoes['train']['score'][0]

4

In [12]:
dados_redacoes['train'].to_pandas()

,essay,score
0,A convivência com o ensino em uma escola públi...,4
1,O Brasil possui aproximadamente 425 mil polici...,5
2,"O esporte é muito importante, as Olimpíadas, u...",8
3,"O conceito de ""banalidade do mal"" da filósofa ...",8
4,Um dos assuntos que mais geram discussões atua...,5
...,...,...
4565,"Com o início da era digital, a capacidade de t...",6
4566,"O feminicídio é o assassinato de uma mulher, a...",4
4567,Foram vrias as conquistas que as mulheres bras...,5
4568,Segundo o psicoterapeuta Ivan Carpelatto: “a v...,9


## Separar Dados

In [13]:
treino_teste = dados_redacoes['train'].train_test_split(test_size = 0.2, shuffle = False)
treino_teste

DatasetDict({
    train: Dataset({
        features: ['essay', 'score'],
        num_rows: 3656
    })
    test: Dataset({
        features: ['essay', 'score'],
        num_rows: 914
    })
})

In [14]:
from datasets import DatasetDict
dados_redacoes = DatasetDict({
    'treino': treino_teste['train'],
    'teste': treino_teste['test']
})


In [15]:
dados_redacoes

DatasetDict({
    treino: Dataset({
        features: ['essay', 'score'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['essay', 'score'],
        num_rows: 914
    })
})

# Tokenizando Dados Textuais

In [16]:
checkpoint_modelo = 'Geotrend/distilbert-base-pt-cased'

In [17]:
from transformers import AutoTokenizer

In [18]:
tokenizador = AutoTokenizer.from_pretrained(checkpoint_modelo)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/162k [00:00<?, ?B/s]

In [19]:
texto = dados_redacoes['treino']['essay'][0]

In [20]:
tokens = tokenizador.tokenize(texto)
print(tokens)

['A', 'con', '##viv', '##ência', 'com', 'o', 'ensino', 'em', 'uma', 'escola', 'pública', ',', 'nos', 'mostra', '\\', 'xa', '##0', '##com', '##o', 'a', 'reforma', 'do', 'ensino', 'médio', 'é', 'ne', '##ces', '##s', '##ária', ',', 'com', 'o', 'ensino', 'pre', '##c', '##ário', ',', 'escolas', 'quase', 'desa', '##band', '##o', ',', 'falta', 'de', 'edu', '##cado', '##res', ',', 'a', 'formação', 'de', 'estudantes', 'fica', 'a', '\\', 'xa', '##0', '##cada', 'dia', 'mais', 'difícil', '.', ',', 'Atualmente', 'o', 'índice', 'de', 'jovens', 'que', 'procura', '##m', 'uma', 'educação', 'de', 'qualidade', 'é', 'maior', 'do', 'que', 'em', 'todos', 'os', 'séculos', ',', 'tem', '##os', 'em', 'vista', 'que', 'todos', 'procura', '##m', 'a', 'melhor', '##ia', ',', 'tanto', 'na', 'estrutura', 'da', 'instituição', 'como', 'no', 'ensino', ',', 'muitos', 'son', '##ham', 'em', 'ser', ':', 'médicos', ',', 'professore', '##s', ',', 'ad', '##vog', '##ados', ',', 'conta', '##dores', ',', 'e', 'assim', 'su', '##ces

In [21]:
ids = tokenizador.convert_tokens_to_ids(tokens)
print(ids)

[46, 270, 24522, 2475, 303, 91, 13849, 348, 469, 5148, 5155, 25, 1125, 4201, 73, 5308, 759, 5563, 231, 77, 8008, 246, 13849, 19640, 144, 539, 2597, 206, 4354, 25, 303, 91, 13849, 1480, 415, 2854, 25, 16687, 8774, 3151, 4948, 231, 25, 5988, 203, 5725, 8294, 920, 25, 77, 11569, 203, 22157, 13510, 77, 73, 5308, 759, 6073, 608, 578, 7859, 27, 25, 12242, 91, 20373, 203, 18423, 220, 21237, 244, 469, 17937, 203, 16944, 144, 2695, 246, 220, 348, 1692, 463, 16426, 25, 1802, 383, 348, 3416, 220, 1692, 21237, 244, 77, 5915, 359, 25, 1813, 230, 9885, 240, 21600, 314, 286, 13849, 25, 6455, 385, 1948, 348, 510, 39, 17306, 25, 17575, 206, 25, 712, 22852, 2092, 25, 5719, 2669, 25, 81, 4832, 292, 2597, 12125, 576, 27, 25, 46, 2996, 10877, 4245, 77, 10673, 15832, 203, 469, 2900, 348, 955, 258, 441, 22157, 25, 15218, 2041, 510, 465, 17157, 25, 77, 4567, 441, 18423, 144, 77, 4567, 246, 5343, 25, 15894, 16539, 91, 13849, 3121, 25, 15894, 16539, 21388, 348, 16687, 13027, 25, 16687, 4577, 9885, 317, 81, 2339

In [22]:
texto_decodificado = tokenizador.decode(ids)
print(texto_decodificado)

A convivência com o ensino em uma escola pública, nos mostra \ xa0como a reforma do ensino médio é necessária, com o ensino precário, escolas quase desabando, falta de educadores, a formação de estudantes fica a \ xa0cada dia mais difícil., Atualmente o índice de jovens que procuram uma educação de qualidade é maior do que em todos os séculos, temos em vista que todos procuram a melhoria, tanto na estrutura da instituição como no ensino, muitos sonham em ser : médicos, professores, advogados, contadores, e assim sucessivamente., A solução seria a constatação de uma lei em prol dos estudantes, precisamos ser ouvidos, a voz dos jovens é a voz do futuro, queremos o ensino superior, queremos oportunidades em escolas públicas, escolas bem estruturada e profissionais competentes., Creio \ xa0que a reforma deveria ocorrer, pois o intuito é a melhoria ( Que tipo de melhoria?, \ xa0assim todos os jovens teriam a oportunidade de serem profissionais de sucesso. O incentivo ( aos jovens é o que to

In [23]:
input_codificado = tokenizador(texto, return_tensors = 'tf')
print(input_codificado)

{'input_ids': <tf.Tensor: shape=(1, 271), dtype=int32, numpy=
array([[   11,    46,   270, 24522,  2475,   303,    91, 13849,   348,
          469,  5148,  5155,    25,  1125,  4201,    73,  5308,   759,
         5563,   231,    77,  8008,   246, 13849, 19640,   144,   539,
         2597,   206,  4354,    25,   303,    91, 13849,  1480,   415,
         2854,    25, 16687,  8774,  3151,  4948,   231,    25,  5988,
          203,  5725,  8294,   920,    25,    77, 11569,   203, 22157,
        13510,    77,    73,  5308,   759,  6073,   608,   578,  7859,
           27,    25, 12242,    91, 20373,   203, 18423,   220, 21237,
          244,   469, 17937,   203, 16944,   144,  2695,   246,   220,
          348,  1692,   463, 16426,    25,  1802,   383,   348,  3416,
          220,  1692, 21237,   244,    77,  5915,   359,    25,  1813,
          230,  9885,   240, 21600,   314,   286, 13849,    25,  6455,
          385,  1948,   348,   510,    39, 17306,    25, 17575,   206,
           25, 

## Tokenização de Toda Base de Dados

In [24]:
def funcao_tokenizadora(dados_texto):
    return tokenizador(dados_texto['essay'], truncation = True)

In [25]:
dataset_tokenizado = dados_redacoes.map(funcao_tokenizadora, batched = True, remove_columns = ['essay'])

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

In [26]:
dataset_tokenizado

DatasetDict({
    treino: Dataset({
        features: ['score', 'input_ids', 'attention_mask'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['score', 'input_ids', 'attention_mask'],
        num_rows: 914
    })
})

In [27]:
dataset_tokenizado['treino'].to_pandas()

,score,input_ids,attention_mask
0,4,"[11, 46, 270, 24522, 2475, 303, 91, 13849, 348...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,5,"[11, 60, 1497, 6656, 5124, 9019, 2782, 22218, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,8,"[11, 60, 290, 8465, 144, 3240, 1668, 25, 243, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,8,"[11, 60, 19131, 203, 15, 23380, 19311, 246, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5,"[11, 2567, 441, 17363, 206, 220, 578, 2989, 78...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
3651,5,"[11, 46, 5148, 1802, 469, 17870, 7326, 230, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3652,6,"[11, 60, 7234, 1280, 3543, 1802, 91, 2885, 203...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3653,4,"[11, 7855, 325, 4776, 4323, 1559, 14086, 13232...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3654,6,"[11, 60, 23602, 235, 326, 23949, 206, 203, 179...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [28]:
dataset_tokenizado['teste'].to_pandas()

,score,input_ids,attention_mask
0,7,"[11, 60, 9133, 18342, 2294, 220, 13606, 1295, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,8,"[11, 599, 3088, 1306, 25, 22568, 2284, 2251, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,4,"[11, 46, 15182, 20233, 16866, 2969, 14150, 81,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,7,"[11, 60, 10910, 303, 469, 7592, 1828, 21261, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,7,"[11, 1876, 91, 3222, 240, 1774, 9151, 359, 25,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
909,6,"[11, 3444, 91, 5764, 240, 452, 3543, 25, 77, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
910,4,"[11, 60, 4880, 1235, 2011, 3575, 144, 91, 2370...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
911,5,"[11, 24848, 98, 8544, 243, 6634, 206, 220, 243...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
912,9,"[11, 4433, 91, 22999, 22429, 14533, 3507, 304,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


# Ajustando a Variavel Alvo - NOTA

In [29]:
dataset_tokenizado['treino'].features['score']

Value(dtype='int64', id=None)

In [30]:
from datasets import ClassLabel

In [31]:
dataset_tokenizado['treino'].unique('score')

[4, 5, 8, 6, 10, 7, 1, 9, 3, 0, 2]

In [32]:
scores = ClassLabel(names = [str(i) for i in range(11)])

In [33]:
scores

ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], id=None)

## Mapeando para nossos dados

In [34]:
def mapear_labels(dados):
    dados['label'] = scores.str2int(str(dados['score']))
    return dados

In [35]:
dataset_tokenizado = dataset_tokenizado.map(mapear_labels, remove_columns = 'score')

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

In [36]:
dataset_tokenizado = dataset_tokenizado.cast_column('label', scores)

Casting the dataset:   0%|          | 0/3656 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/914 [00:00<?, ? examples/s]

In [37]:
dataset_tokenizado

DatasetDict({
    treino: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 914
    })
})

In [38]:
dataset_tokenizado['treino'].features['label']

ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], id=None)

# Carregando o modelo pré-treinado

In [39]:
from transformers import TFAutoModelForSequenceClassification

In [40]:
checkpoint_modelo = 'Geotrend/distilbert-base-pt-cased'

In [41]:
id2label = {0: '0', 1: '1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9',10:'10'}
label2id = {valor: chave for chave, valor in id2label.items()}

modelo = TFAutoModelForSequenceClassification.from_pretrained(
    checkpoint_modelo,
    num_labels = dataset_tokenizado['treino'].features['label'].num_classes,
    id2label = id2label,
    label2id = label2id,
    from_pt = True
)


pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

In [42]:
dados_treino = modelo.prepare_tf_dataset(
    dataset_tokenizado['treino'],
    shuffle = True,
    batch_size = 16,
    tokenizer = tokenizador
)

dados_validacao = modelo.prepare_tf_dataset(
    dataset_tokenizado['teste'],
    shuffle = False,
    batch_size = 16,
    tokenizer = tokenizador
)

# Treinando Modelo

Criar um otimizador que ajusta o modelo aos dados de treinamento e esse otimizador utilizará o conhecimento que já foi adquirido pelo modelo pré-treinado.

In [43]:
from transformers import create_optimizer

In [44]:
batch_size = 16
epocas = 10
batches_por_epoca = len(dataset_tokenizado['treino']) // batch_size
total_passos_treino = int(batches_por_epoca * epocas)
taxa_aprendizado = 2e-5

In [45]:
otimizador, scheduler = create_optimizer(
    init_lr = taxa_aprendizado, num_warmup_steps = 0, num_train_steps = total_passos_treino
)

modelo.compile(optimizer = otimizador)

In [46]:
modelo.fit(dados_treino, validation_data = dados_validacao, epochs= epocas)

Epoch 1/10
228/228 [==============================] - 235s 933ms/step - loss: 1.9039 - val_loss: 1.7438
Epoch 2/10
228/228 [==============================] - 219s 961ms/step - loss: 1.6565 - val_loss: 1.5763
Epoch 3/10
228/228 [==============================] - 218s 957ms/step - loss: 1.5192 - val_loss: 1.5960
Epoch 4/10
228/228 [==============================] - 221s 971ms/step - loss: 1.3948 - val_loss: 1.5862
Epoch 5/10
228/228 [==============================] - 218s 957ms/step - loss: 1.2555 - val_loss: 1.5717
Epoch 6/10
228/228 [==============================] - 219s 960ms/step - loss: 1.1172 - val_loss: 1.6688
Epoch 7/10
228/228 [==============================] - 221s 972ms/step - loss: 0.9686 - val_loss: 1.7318
Epoch 8/10
228/228 [==============================] - 218s 958ms/step - loss: 0.8341 - val_loss: 1.8096
Epoch 9/10
228/228 [==============================] - 218s 958ms/step - loss: 0.7628 - val_loss: 1.8707
Epoch 10/10
228/228 [==============================] - 219s 959m

In [47]:
resultados_avaliacao = modelo.evaluate(dados_validacao)
print(f'Loss: {resultados_avaliacao}')

58/58 [==============================] - 17s 296ms/step - loss: 1.8872
Loss: 1.8872305154800415


# Salvando e Publicando Modelo

In [48]:
from huggingface_hub import notebook_login

notebook_login()

In [49]:
modelo.push_to_hub('distilbert-pt-cased-redacao-nota')

tf_model.h5:   0%|          | 0.00/251M [00:00<?, ?B/s]

In [50]:
tokenizador.push_to_hub('distilbert-pt-cased-redacao-nota')

README.md:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MagnoMorgan/distilbert-pt-cased-redacao-nota/commit/c597983734c06ea711589c61f61d1f6067334696', commit_message='Upload tokenizer', commit_description='', oid='c597983734c06ea711589c61f61d1f6067334696', pr_url=None, pr_revision=None, pr_num=None)

# Utilizando o modelo para previsão

## Carregando Modelo

In [51]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [52]:
modelo = TFAutoModelForSequenceClassification.from_pretrained('MagnoMorgan/distilbert-pt-cased-redacao-nota')
tokenizador = AutoTokenizer.from_pretrained('MagnoMorgan/distilbert-pt-cased-redacao-nota')

config.json:   0%|          | 0.00/901 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some layers from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/162k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/554k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [53]:
dados_redacoes['teste'].to_pandas()

,essay,score
0,O atual vírus que expandiu o Brasil está sendo...,7
1,"No século XIX, Positivismo defendia o uso do m...",8
2,A Qualificação qualificação e o futuro do empr...,4
3,O homem com uma arma nas mãos é um bicho feroz...,7
4,"Durante o período da Guerra Fria, ocorreu o ma...",7
...,...,...
909,"Com o início da era digital, a capacidade de t...",6
910,"O feminicídio é o assassinato de uma mulher, a...",4
911,Foram vrias as conquistas que as mulheres bras...,5
912,Segundo o psicoterapeuta Ivan Carpelatto: “a v...,9


In [54]:
textos = [dados_redacoes['teste']['essay'][2],
          dados_redacoes['teste']['essay'][909]]

In [55]:
import numpy as np

In [56]:
textos_tokenizados = tokenizador(textos, return_tensors = 'np', padding = 'longest')

In [57]:
resultados = modelo(textos_tokenizados).logits
resultados

<tf.Tensor: shape=(2, 11), dtype=float32, numpy=
array([[-1.04678  , -2.7735083, -1.6387925, -1.0142949,  1.4793586,
         4.084616 ,  2.7290335, -0.7963695, -1.8280723, -2.6807072,
        -2.582347 ],
       [-2.2096741, -3.8440154, -3.3406372, -2.542177 , -1.6281905,
         0.3464645,  3.8633387,  4.0632024,  0.5859413, -1.7762108,
        -2.1574924]], dtype=float32)>

In [58]:
classificacao = np.argmax(resultados, axis = 1)
print(classificacao)

[5 7]


## Automatizando Processo

In [59]:
from transformers import pipeline

classificador = pipeline('text-classification', 'MagnoMorgan/distilbert-pt-cased-redacao-nota', framework = 'tf')

Some layers from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerat

In [60]:
classificador(textos)

[{'label': '5', 'score': 0.7348980903625488},
 {'label': '7', 'score': 0.5300719738006592}]

# Interface Gradio

In [61]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.1 MB/s eta 0:00:00


In [62]:
import gradio as gr
from transformers import pipeline

In [65]:
classificador = pipeline('text-classification', 'MagnoMorgan/distilbert-pt-cased-redacao-nota', framework = 'tf')

Some layers from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at MagnoMorgan/distilbert-pt-cased-redacao-nota and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerat

In [66]:
def mostrar_resultado(texto):
    if len(texto) < 400:
        return 'O texto precisa ter no mínimo 400 caracteres'
    return int(classificador(texto)[0]['label']) * 100


In [67]:
app = gr.Interface(
    fn = mostrar_resultado,
    inputs = gr.Textbox(label = 'Digite o texto'),
    outputs = gr.Textbox(label = 'Nota da redação'),
    title = 'Insira um texto para receber uma nota entre 0 e 1000'
)

app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d594a7ec514851952e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
